In [1]:
#pip和环境（需要修改模型地址）

import subprocess
# 安装包
subprocess.run(['pip', 'install', 'adversarial-robustness-toolbox'])
subprocess.run(['pip', 'install', 'pandas'])
subprocess.run(['pip', 'install', 'numpy'])
#模型地址
resnet_name=r"D:\python\resnetcifar\model\resmodel.pth"

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from art.estimators.classification import PyTorchClassifier
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim

# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
])

# 加载CIFAR-10数据集
trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=transform)

# 合并训练集和测试集
combined_data = trainset.data
combined_labels = trainset.targets
combined_data = np.concatenate((combined_data, testset.data), axis=0)
combined_labels = np.concatenate((combined_labels, testset.targets), axis=0)

# 获取类别名称
class_names = trainset.classes

# 创建一个字典来存储每个类别的DataFrame
dataframes = {}

# 按标签分组数据
for label in range(len(class_names)):
    # 获取当前标签的索引
    indices = np.where(combined_labels == label)[0]
    
    # 提取对应的数据和标签
    label_data = combined_data[indices]
    label_labels = combined_labels[indices]
    
    # 创建DataFrame
    df = pd.DataFrame({
        'image': [img for img in label_data],
        'label': label_labels
    })
    
    # 用类别名称命名DataFrame
    dataframes[class_names[label]] = df

# 返回10个DataFrame



# 如果需要返回这些DataFrame，可以返回字典
# return dataframes

In [ ]:
import torchvision.models as models
# 定义与保存时一致的模型结构
class ModifiedResNet18(torch.nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedResNet18, self).__init__()
        self.resnet18 = models.resnet18(pretrained=False)  # 不使用预训练权重，因为我们加载的是已经训练好的模型
        self.resnet18.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet18.maxpool = torch.nn.Identity()  # 移除最大池化层

        # 获取ResNet18的特征提取部分
        self.features = torch.nn.Sequential(*list(self.resnet18.children())[:-1])

        # 添加MLP和全连接层
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5)
        )
        self.fc = torch.nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.mlp(x)
        x = self.fc(x)
        return x

# 初始化模型
model = ModifiedResNet18()

# 加载保存的模型权重
model_path = resnet_name # 替换为你的模型文件路径
model.load_state_dict(torch.load(model_path))


In [ ]:
def create_art_classifier(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(0, 1),
        loss=criterion,
        optimizer=optimizer,
        input_shape=(3, 32, 32),
        nb_classes=10,
    )
    return classifier
classifier=create_art_classifier(model)

In [ ]:
#AutoAttack攻击
from art.attacks.evasion import AutoAttack
def attack_dataframe_autoattack(df, epsilon=8/255):
    # 创建ART分类器
    classifier = create_art_classifier(model)
    # 创建AutoAttack攻击对象
    attack = AutoAttack(
        estimator=classifier,
        norm=np.inf,  # 使用np.inf表示L∞范数
        eps=epsilon,
        eps_step=epsilon / 4,
        targeted=False,
        batch_size=512
    )

    # 创建一个新的DataFrame存储攻击后的图像数据
    attacked_df = pd.DataFrame(columns=['image', 'label'])

    # 遍历输入的DataFrame中的每张图像
    for index, row in df.iterrows():
        # 提取图像和标签
        image = row['image']
        label = row['label']

        # 将图像转换为Tensor并归一化到[0, 1]
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0) / 255.0

        # 生成对抗样本
        perturbed_image = attack.generate(x=image_tensor.numpy())

        # 将攻击后的图像转换回NumPy数组
        attacked_image = (perturbed_image.squeeze(0) * 255).astype(np.uint8)

        # 将攻击后的图像和标签添加到新的DataFrame中
        new_row = pd.DataFrame({'image': [attacked_image], 'label': [label]})
        attacked_df = pd.concat([attacked_df, new_row], ignore_index=True)

        # 打印进度
        if (index + 1) % 1000 == 0:
            print(f"Processed {index + 1} images")

    return attacked_df

In [ ]:
# 对整个字典中的所有类别进行AutoAttack攻击
def attack_all_dataframes_autoattack(dataframes, epsilon=8/255):
    attacked_dataframes = {}
    for class_name, df in dataframes.items():
        print(f"Attacking class: {class_name}")
        attacked_df = attack_dataframe_autoattack(df, epsilon)
        attacked_dataframes[class_name] = attacked_df
        
                # 创建一个新的DataFrame来存储攻击后的数据
        df_to_save = attacked_df.copy()
        df_to_save['class_name'] = class_name  # 添加类别名称列

        # 将图像数据转换为字符串格式以便保存
        df_to_save['image'] = df_to_save['image'].apply(lambda x: x.tobytes())

        # 保存到CSV文件
        filename = f"{class_name}_auto.csv"
        df_to_save.to_csv(filename, index=False)
        print(f"Data for class '{class_name}' saved to {filename}")
    return attacked_dataframes

In [ ]:
# 删除airplane这一类
if 'airplane' in dataframes:
    del dataframes['airplane']
    print("Deleted 'airplane' class from dataframes.")
else:
    print("'airplane' class not found in dataframes.")
if 'automobile' in dataframes:
    del dataframes['automobile']
    print("Deleted automobile class from dataframes.")  
autos=attack_all_dataframes_autoattack(dataframes, epsilon=8/255)